In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    import os
    import sys
    os.chdir('/content/drive/MyDrive/DL_Project_GD/')
    sys.path.append('/content/drive/MyDrive/DL_Project_GD/code/')

    #%% [code]
    ! pip install -qqq "torchvision" "torch>=1.6, <1.9" "torchmetrics>=0.3" "ipython[notebook]" "pytorch-lightning>=1.3" "torchtext"
    ! pip install -qqq cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

    ## Install dependencies
    ! pip install -qqq wget
    ! apt-get -qqq install sox libsndfile1 ffmpeg
    ! pip install -qqq unidecode
    ! pip install -qqq matplotlib>=3.3.2

    ## Install NeMo
    BRANCH = 'r1.6.1'
    ! python -m pip install -qqq git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

    ! mkdir configs
    ! wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml

    datapath = './SyncData/wav/'
    rootpath = './'
    print("Running with Colab setup.")
    tpu_cores = 8
    gpu_cores = None
except(ModuleNotFoundError):
    print('Initialising non-colab set-up...')
    tpu_cores=None
    gpu_cores=None
    datapath = '/Users/jameswilkinson/Downloads/dev/aac/'
    rootpath = '../'

from VGGmini import VGGmini
from dataloader import VoxDataloader
import runner
import os
print("cwd: {}".format(os.getcwd()))
print("datapath: {}".format(datapath))
print('done.')

In [ ]:
config = {
            # learning params
            'lr': 1e-3,
            'batch_size': 64,
            'early_stopping': True,
            'optimizer': 'Adam',
            'max_epochs': 20,
            'early_stopping': True,
            'patience': 3, # tolerance for early stopping
            'momentum': 0.,
            'lr_decay': 0.,

            # model
            'fftmethod': 'librosa.stft', # choose librosa.stft, signal.stft, or librosa.mfcc
            'pooling' : 'average', # 'average' or 'max'. will not have effect if not using VGGpool

            #, regularization
            'dropout': 0.0,
            'L2': 0.0,
            'batch_norm': True,
            'gradient_clipping': 0.0,
}

In [6]:
for method in ['librosa.stft', 'librosa.mel', 'librosa.mfcc','pydct.sdct']:
  config['fftmethod'] = method
  runner.run(config, datapath, VoxDataloader, VGGmini, './VGGmini/', tpu_cores=tpu_cores, gpu_cores=gpu_cores, phase_map_file='phase_map_40.csv')


LOGNAME: spctlibrosa.stft_optAdam_lr0.001_reg0.0_drop0.0_bnTrue_gc0.0_mom0.0_dec0.0_poolingaverage


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Missing logger folder: ./VGGmini/spctlibrosa.stft_optAdam_lr0.001_reg0.0_drop0.0_bnTrue_gc0.0_mom0.0_dec0.0_poolingaverage

   | Name       | Type             | Params
-------------------------------------------------
0  | loss       | CrossEntropyLoss | 0     
1  | activation | ReLU             | 0     
2  | conv1      | Conv2d           | 4.8 K 
3  | mpool1     | MaxPool2d        | 0     
4  | conv2      | Conv2d           | 2.5 M 
5  | mpool2     | MaxPool2d        | 0     
6  | fc3        | Linear           | 32    
7  | fc4        | Linear           | 18    
8  | fc5        | Linear           | 35.9 K
9  | seq1       | Sequential       | 192   
10 | seq2       | Sequential       | 2.0 K 
11 | seq3       | Sequential       | 2.0 K 
-------------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
10.015    To

N classes: 35


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/jameswilkinson/.conda/envs/DL_Project/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/gq/c66p8yrd3bd1sr7svmy6k91h0000gn/T/ipykernel_39043/3373228469.py", line 3, in <module>
    runner.run(config, datapath, VoxDataloader, VGGmini, './VGGmini/', tpu_cores=tpu_cores, gpu_cores=gpu_cores, phase_map_file='phase_map_40.csv')
  File "/Users/jameswilkinson/Documents/python programs/DL_Project/code/runner.py", line 44, in run
  File "/Users/jameswilkinson/.conda/envs/DL_Project/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 740, in fit
    self._call_and_handle_interrupt(
  File "/Users/jameswilkinson/.conda/envs/DL_Project/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py", line 693, in _call_and_handle_interrupt
    self.on_exception(exception)
KeyboardInterrupt

During handling of the above 

TypeError: object of type 'NoneType' has no len()